In [27]:
import tensorflow as tf
import numpy as np

In [28]:
bandits = [0.2, 0, -0.2, -5]  # lower bandit number, higher chance of positive reward
num_bandits = len(bandits)

In [29]:
#reward mechanism

def pullBandit(bandit):
    #Get a random number
    result = np.random.randn(1)
    if result > bandit:
        return 1  # positive reward
    else:
        return -1 #negative reward

In [30]:
np.random.rand(1)

array([ 0.04469142])

In [34]:
tf.reset_default_graph()

In [35]:
# feed forward net
weights = tf.Variable(tf.ones([num_bandits]))  # each weight corresponds to one bandit
                                               # represnets how good it is to pull that arm and do that action
chosen_action = tf.argmax(weights,0)     #action with highest weight

# feed reward and chosen action to compute loss
reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)  # reward from selected action
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)  #index of chosen action
responsible_weight = tf.slice(weights, action_holder, [1]) #find weight of chosen action from weights vector
loss = -(tf.log(responsible_weight)*reward_holder)  # Log(policy)*A - compute policy gradient loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001) 
update = optimizer.minimize(loss)

In [36]:
#training the agent

total_episodes = 1000
total_reward = np.zeros(num_bandits) # scoreboard of bandits started at 0
e = 0.1 #chance of taking a random action

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        # choose random action or one with heighest weights with e-greedy prob
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
        # implement action to get reward
        reward = pullBandit(bandits[action]) #get reward from picking a bandit
        
        #update the network, pass in reward and selected action
        _, resp, ww = sess.run([update, responsible_weight, weights], feed_dict={reward_holder:[reward],
                                                                                action_holder:[action]})
        
        # update scores - add reward of that action to total scoreboard
        total_reward[action] += reward
        
        if i%50==0:
            print("Running reward for the "+str(num_bandits)+" bandits: "+str(total_reward))
            print(ww)
        
        i+=1
print("the agent thinks most promising bandit is",str(np.argmax(ww)+1))
            

Running reward for the 4 bandits: [-1.  0.  0.  0.]
[ 1.  1.  1.  1.]
Running reward for the 4 bandits: [-1.  1.  5.  2.]
[ 0.99900109  1.00100613  1.00599766  1.00199902]
Running reward for the 4 bandits: [ -1.   0.  22.   2.]
[ 0.9990021   1.00000715  1.02277887  1.00199902]
Running reward for the 4 bandits: [ -2.  -3.  21.   3.]
[ 0.9980011   0.99700415  1.02084303  1.00199902]
Running reward for the 4 bandits: [ -3.   0.  31.   3.]
[ 0.9980011   1.00001013  1.0306133   1.00299704]
Running reward for the 4 bandits: [ -3.   1.  30.   9.]
[ 0.99699908  1.00101018  1.02869022  1.0089643 ]
Running reward for the 4 bandits: [ -3.   0.  33.   9.]
[ 0.99699908  1.00001121  1.03356147  1.0089643 ]
Running reward for the 4 bandits: [ -3.  -2.  37.   9.]
[ 0.99699908  0.99801022  1.03551614  1.0089643 ]
Running reward for the 4 bandits: [ -3.  -2.  43.  11.]
[ 0.9970001   0.99801022  1.04323339  1.01094556]
Running reward for the 4 bandits: [ -2.  -2.  48.  11.]
[ 0.99800313  0.99801022  1.04

In [23]:
t= np.ones([4])

In [24]:
np.argmax(t,axis=0)

0

In [26]:
np.random.rand(4)*2-1

array([-0.29588233,  0.15985818, -0.27136887,  0.20369762])